In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(setence):
    return nltk.word_tokenize(setence)

In [3]:
tokenize('how are you ????')

['how', 'are', 'you', '?', '?', '?', '?']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['lol','testing','programming','vr'],['lol','testing','programming','vr'])

array([1., 0., 0., 1.], dtype=float32)

In [8]:
data = pd.read_csv('./data.csv')

In [9]:
X = data['text']
y = data['target']

In [10]:
all_words = []
all_data = []
tags = []

In [11]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7613it [00:02, 3080.79it/s]


In [12]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [13]:
all_words = sorted(set(all_words))

In [14]:
tags = sorted(set(tags))

In [15]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)

In [16]:
X = []
y = []

In [17]:
for sent,tag in tqdm(all_data):
    bag = bag_of_words(sent,all_words)
    X.append(bag)
    tag = tags.index(tag)
    y.append(tag)

100%|██████████| 7613/7613 [00:25<00:00, 303.50it/s]


In [18]:
from sklearn.model_selection import *

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,shuffle=False)

In [21]:
device = 'cuda'

In [22]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [23]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [24]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [25]:
class Model(Module):
    def __init__(self,hidden):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.linear3 = Linear(hidden,hidden)
        self.linear4 = Linear(hidden,hidden)
        self.linear5 = Linear(hidden,hidden)
        self.output = Linear(hidden,1)
        self.output_ac = Sigmoid()
    
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output_ac(self.output(preds))
        return preds

In [26]:
device = 'cuda'

In [27]:
model = Model(512).to(device)
model.to(device)

Model(
  (activation): ReLU()
  (linear1): Linear(in_features=13928, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=512, bias=True)
  (linear4): Linear(in_features=512, out_features=512, bias=True)
  (linear5): Linear(in_features=512, out_features=512, bias=True)
  (output): Linear(in_features=512, out_features=1, bias=True)
  (output_ac): Sigmoid()
)

In [28]:
criterion = BCELoss()

In [29]:
optimizer = Adam(model.parameters(),lr=0.001)

In [30]:
epochs = 1000

In [31]:
batch_size = 32

In [32]:
PROJECT_NAME = 'NLP-Disaster-Tweets-Clf-4'

In [33]:
import wandb

In [34]:
wandb.init(project=PROJECT_NAME,name='baseline')
wandb.watch(model)
for _ in tqdm(range(epochs)):
    for idx in range(0,len(X_train),batch_size):
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    wandb.watch(model)
wandb.watch(model)
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


 73%|███████▎  | 731/1000 [28:14<10:23,  2.32s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-d7d408b89cbe>", line 11, in <module>
    loss.backward()
  File "/home/indika/anaconda3/lib/python3.7/site-packages/torch/tensor.py", line 221, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/home/indika/anaconda3/lib/python3.7/site-packages/torch/autograd/__init__.py", line 132, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/indika/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handl

TypeError: object of type 'NoneType' has no len()

Error in callback <function _WandbInit._pause_backend at 0x7f3e6617e830> (for post_run_cell):


KeyboardInterrupt: 

In [ ]:
torch.argmax(preds)

In [ ]:
y_batch